In [1]:
import torch
import torchvision
import numpy as np
from utils import compute_metrics, sequence_loss, InputPadder
from presets import OpticalFlowPresetTrain, OpticalFlowPresetEval
from model import FlowNetS
from multiscaleloss import multiscaleEPE, realEPE
import time
import os

/home/ubuntu/anaconda3/envs/cs231n/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/cs231n/util.py:11: ImportWarning: failed to load custom correlation modulewhich is needed for FlowNetC
  warnings.warn("failed to load custom correlation module"


In [2]:
model_type = "raft"
pretrain_size = 10
finetune_size = 520
pretrain=False

In [3]:
def get_data(pretrain_size, finetune_size):
    fc_transforms = OpticalFlowPresetTrain(crop_size=(368, 496), min_scale=0.1, max_scale=1.0, do_flip=True)
    flying_chairs = torchvision.datasets.FlyingChairs(root=".", split="train", transforms=fc_transforms)
    s_transforms = OpticalFlowPresetTrain(crop_size=(368, 768), min_scale=-0.2, max_scale=0.6, do_flip=True)
    sintel_train = torchvision.datasets.Sintel(root=".", split="train", pass_name="clean", transforms=s_transforms)
    test_transforms = OpticalFlowPresetEval()
    sintel_test = torchvision.datasets.Sintel(root=".", split="train", pass_name="clean", transforms=test_transforms)

    train_ind = np.random.choice(len(sintel_train), finetune_size, replace=False)[:10]
    test_ind = np.array(list(set(range(len(sintel_train))) - set(train_ind)))[:50]
    fc_ind = np.random.choice(len(flying_chairs), pretrain_size, replace=False)

    sintel_train = torch.utils.data.Subset(sintel_train, train_ind)
    sintel_test = torch.utils.data.Subset(sintel_test, test_ind)
    fc_pretrain = torch.utils.data.Subset(flying_chairs, fc_ind)

    train_loader = torch.utils.data.DataLoader(sintel_train, batch_size=10, shuffle=True)
    test_loader = torch.utils.data.DataLoader(sintel_test, batch_size=10, shuffle=False)
    fc_loader = torch.utils.data.DataLoader(fc_pretrain, batch_size=10, shuffle=True)

    return train_loader, test_loader, fc_loader

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
def train_flownet_one_epoch(model, train_loader, optimizer, scheduler, device, epoch):
    start = time.time()
    epoch_loss = 0.0
    for i, data_blob in enumerate(train_loader):
        optimizer.zero_grad()
        
        image1, image2, flow_gt, valid_flow_mask = (x.to(device) for x in data_blob)
        image = torch.cat((image1, image2), dim=1)
        
        output = model(image)

        loss = multiscaleEPE(output, flow_gt)
        loss.backward()
        
        optimizer.step()
        
        # Compute epe loss 
        h, w = flow_gt.size()[-2:]
        upsampled_output = torch.nn.functional.interpolate(output[0], (h,w), mode='bilinear', align_corners=False)
        metrics, _ = compute_metrics(upsampled_output, flow_gt)
        epoch_loss += metrics["epe"]
            
    scheduler.step()
    epoch_loss /= len(train_loader)
    print("Epoch", epoch + 1, "finished in", round(time.time() - start, 1), "seconds. Loss:", epoch_loss)
    return epoch_loss

In [6]:
def train_flownet(fc_loader, train_loader, pretrain=True):
    pretrain_epochs = 100
    finetune_epochs = 20
    lr = 1e-4
    weight_decay = 4e-4
    model = FlowNetS()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    model = model.to(device)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75, 100, 110], gamma=0.5)

    model.train()
    pretrain_losses = []
    finetune_losses = []
    if pretrain:
        print("Pretraining FlowNet on", len(fc_loader.dataset), "FlyingChairs examples...")
        for epoch in range(pretrain_epochs):
            pretrain_losses.append(train_flownet_one_epoch(model, fc_loader, optimizer, scheduler, device, epoch))
    print("Finetuning Flownet on", len(train_loader.dataset), "Sintel examples...")
    for epoch in range(finetune_epochs):
        finetune_losses.append(train_flownet_one_epoch(model, train_loader, optimizer, scheduler, device, epoch))
    return model, pretrain_losses, finetune_losses

In [7]:
def test_flownet(model, test_loader):
    model.eval()
    total_epe = 0.0
    total_f1 = 0.0

    with torch.no_grad():
        for i, data_blob in enumerate(test_loader):
            image1, image2, flow_gt, valid_flow_mask = (x.to(device) for x in data_blob)
            image = torch.cat((image1, image2), dim=1)

            output = model(image)
            h, w = flow_gt.size()[-2:]
            upsampled_output = torch.nn.functional.interpolate(output, (h,w), mode='bilinear', align_corners=False)
            metrics, _ = compute_metrics(upsampled_output, flow_gt, valid_flow_mask)
            epe = metrics["epe"]
            f1 = metrics["f1"]

            total_epe += epe
            total_f1 += f1

    total_epe /= len(test_loader)
    total_f1 /= len(test_loader)
    
    return total_epe, total_f1

In [8]:
def train_raft(fc_loader, train_loader, pretrain=True):
    model = torchvision.models.optical_flow.raft_small()
    model.to(device)
    pretrain_epochs = 20
    finetune_epochs = 10

    lr = 2e-5
    weight_decay = 5e-5
    eps = 1e-8
    num_train_flow_updates = 12
    
    total_steps = finetune_epochs * len(train_loader)
    if pretrain:
        total_steps += pretrain_epochs * len(fc_loader)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay, eps=eps)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer=optimizer,
        max_lr=lr,
        total_steps=total_steps,
        pct_start=0.05,
        cycle_momentum=False,
        anneal_strategy="linear",
    )
    
    model.train()
    pretrain_losses = []
    finetune_losses = []
    if pretrain:
        print("Pretraining RAFT on", len(fc_loader.dataset), "FlyingChairs examples...")
        for epoch in range(pretrain_epochs):
            pretrain_losses.append(train_raft_one_epoch(model, fc_loader, optimizer, scheduler, device, epoch, num_train_flow_updates))
    print("Finetuning RAFT on", len(train_loader.dataset), "Sintel examples...")
    for epoch in range(finetune_epochs):
        finetune_losses.append(train_raft_one_epoch(model, train_loader, optimizer, scheduler, device, epoch, num_train_flow_updates))
    return model, pretrain_losses, finetune_losses
    

In [9]:
def train_raft_one_epoch(model, train_loader, optimizer, scheduler, device, epoch, num_train_flow_updates):
    start = time.time()
    epoch_loss = 0.0
    for i, data_blob in enumerate(train_loader):
        optimizer.zero_grad()

        image1, image2, flow_gt, valid_flow_mask = (x.to(device) for x in data_blob)
        flow_predictions = model(image1, image2, num_flow_updates=num_train_flow_updates)

        loss = sequence_loss(flow_predictions, flow_gt, valid_flow_mask)
        metrics, epe_num = compute_metrics(flow_predictions[-1], flow_gt, valid_flow_mask)

        epoch_loss += metrics["epe"]

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        scheduler.step()
    epoch_loss /= len(train_loader)
    print("Epoch", epoch + 1, "finished in", round(time.time() - start, 1), "seconds. Loss:", epoch_loss)
    return epoch_loss

In [10]:
def test_raft(model, test_loader, num_test_updates=32):
    model.eval()
    total_epe = 0.0
    total_f1 = 0.0
    

    with torch.no_grad():
        for i, data_blob in enumerate(test_loader):
            image1, image2, flow_gt = (x.to(device) for x in data_blob)
            padder = InputPadder(image1.shape)
            image1, image2 = padder.pad(image1, image2)

            flow_predictions = model(image1, image2, num_flow_updates=num_test_updates)
            flow_pred = flow_predictions[-1]
            flow_pred = padder.unpad(flow_pred)
            
            metrics, _ = compute_metrics(flow_pred, flow_gt)
            epe = metrics["epe"]
            f1 = metrics["f1"]

            total_epe += epe
            total_f1 += f1

    total_epe /= len(test_loader)
    total_f1 /= len(test_loader)
    
    return total_epe, total_f1

In [11]:
def save_results(results, model_type, pretrain_size):
    path = os.path.join("results", model_type + str(pretrain_size) + "_results.txt")
    torch.save(results, path)

In [12]:
def save_model(model, model_type, pretrain_size):
    path = os.path.join("results", model_type + str(pretrain_size) + "_params.pt")
    torch.save(model.state_dict(), path)

In [13]:
train_loader, test_loader, fc_loader = get_data(pretrain_size, finetune_size)


if model_type == "flownet":    
    model, pretrain_losses, finetune_losses = train_flownet(fc_loader, train_loader, pretrain)
    epe, f1 = test_flownet(model, test_loader)
    print("Test:")
    print("Epe: ", epe)
    print("F1: ", f1)
    results = {
        "pretrain_losses": pretrain_losses, 
        "finetune_losses": finetune_losses,
        "epe": epe,
        "f1": f1
    }
    if not pretrain:
        pretrain_size = 0
    save_results(results, model_type, pretrain_size)
    save_model(model, model_type, pretrain_size)
elif model_type == "raft":
    model, pretrain_losses, finetune_losses = train_raft(fc_loader, train_loader, pretrain)
    epe, f1 = test_raft(model, test_loader)
    print("Test:")
    print("Epe: ", epe)
    print("F1: ", f1)
    results = {
        "pretrain_losses": pretrain_losses, 
        "finetune_losses": finetune_losses,
        "epe": epe,
        "f1": f1
    }
    if not pretrain:
        pretrain_size = 0
    save_results(results, model_type, pretrain_size)
    save_model(model, model_type, pretrain_size)
else:
    raise(Exception("Invalid model type"))

Finetuning RAFT on 10 Sintel examples...
Epoch 1 finished in 4.1 seconds. Loss: 24.054672241210938
Epoch 2 finished in 4.5 seconds. Loss: 23.740591049194336
Epoch 3 finished in 3.9 seconds. Loss: 22.062257766723633
Epoch 4 finished in 4.2 seconds. Loss: 21.060420989990234
Epoch 5 finished in 4.2 seconds. Loss: 20.879302978515625
Epoch 6 finished in 3.9 seconds. Loss: 24.18783187866211
Epoch 7 finished in 4.3 seconds. Loss: 24.239057540893555
Epoch 8 finished in 3.9 seconds. Loss: 21.87493324279785
Epoch 9 finished in 4.0 seconds. Loss: 20.071794509887695
Epoch 10 finished in 4.1 seconds. Loss: 22.1276912689209
Test:
Epe:  30.536666107177734
F1:  98.31702947616577


In [19]:
pretrain_losses

[]

In [21]:
torch.load("results/raft0_results.txt")

{'pretrain_losses': [],
 'finetune_losses': [51.80144500732422,
  43.32882308959961,
  30.19902801513672,
  30.44305992126465,
  28.65836524963379,
  30.460718154907227,
  26.02287483215332,
  23.477127075195312,
  24.884212493896484,
  26.327390670776367],
 'epe': 34.20448265075684,
 'f1': 99.13549184799194}